In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt
import os
os.getcwd()
dir = "/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data/"
os.chdir(dir)
os.getcwd()

'/Users/walterdempsey/Box/MD2K Processed Data/smoking-lvm-cleaned-data'

In [2]:
random_accptresponse = ['1 - 19 Minutes', '20 - 39 Minutes', '40 - 59 Minutes', 
                    '60 - 79 Minutes', '80 - 100 Minutes']
random_dictionary = {'1 - 19 Minutes': 10, 
                     '20 - 39 Minutes': 30, 
                     '40 - 59 Minutes':50,
                     '60 - 79 Minutes':70, 
                     '80 - 100 Minutes':90 } 

In [7]:
# read data
random_original_cloud_ids = [201, 203, 206, 210, 221, 226, 229] 
random_ema = pd.read_csv(dir + 'random-ema.csv')
random_ema = random_ema.drop(['offset'], axis = 1)
random_ema_alternative = pd.read_csv(dir + 'random-ema-alternative.csv')
random_ema_backup = pd.read_csv(dir + 'random-ema-backup.csv')

temp_random_original = random_ema[random_ema['participant_id'].isin(random_original_cloud_ids)]
temp_random_alt = random_ema_alternative[~random_ema_alternative['participant_id'].isin(random_original_cloud_ids)]

random_complete = pd.concat([temp_random_original, temp_random_alt, random_ema_backup])

puffmarker_original_cloud_ids = [201, 203, 206, 210, 221, 229] 
puffmarker = pd.read_csv(dir + 'puff-probability.csv')
puffmarker = puffmarker.drop(['offset'], axis = 1)
puffmarker_alternative = pd.read_csv('puff-probability-alternative.csv')
puffmarker_backup = pd.read_csv('puff-probability-backup.csv')

temp_puffmarker_original = puffmarker[puffmarker['participant_id'].isin(puffmarker_original_cloud_ids)]
temp_puffmarker_alt = puffmarker_alternative[~puffmarker_alternative['participant_id'].isin(puffmarker_original_cloud_ids)]

puffmarker_complete = pd.concat([temp_puffmarker_original, temp_puffmarker_alt, puffmarker_backup])


In [43]:
def random_puff(delta):
    '''
    checks how sensitive is puffmarker coverage of 
    random EMAs to time window
    '''
    # print("Set delta to {} minutes".format(delta))

    htm_complete_yes = []
    htm_complete_no = []
    for id in set(contingent_complete['participant_id']) & set(puffmarker_complete['participant_id']):

        puffmarker_id = np.where(puffmarker_complete['participant_id'] == id) 
        puffmarker_subset = puffmarker_complete.iloc[puffmarker_id[0]]
        try:
            puffmarker_dates_list = [datetime.datetime.strptime(date, '%m/%d/%y %H:%M') for date in puffmarker_subset['date']]
        except:
            puffmarker_dates_list = [datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in puffmarker_subset['date']]

        random_id = np.where(random_complete['participant_id'] == id) 
        random_subset = random_complete.iloc[random_id[0]]

        htm_id_list_yes = []
        htm_id_list_no = []
        for index, row in random_subset.iterrows():
            try:
                random_time = datetime.datetime.strptime(row['date'], '%m/%d/%y %H:%M')
            except:
                random_time = datetime.datetime.strptime(row['date'], '%Y-%m-%d %H:%M:%S')
            if row['when_smoke'] in random_accptresponse:
                random_time = random_time - datetime.timedelta(minutes=random_dictionary[row['when_smoke']])
            if row['when_smoke'] in random_accptresponse: 
                htm_count = 0
                for index_puff in range(0,len(puffmarker_dates_list)):
                    temp = abs((puffmarker_dates_list[index_puff] - random_time).total_seconds() / 60.0)
                    if temp <= delta:
                        htm_count += 1
                htm_id_list_yes.append(htm_count)  
            if row['smoke'] == 'No':
                htm_count = 0
                for index_puff in range(0,len(puffmarker_dates_list)):
                    temp = abs((puffmarker_dates_list[index_puff] - random_time).total_seconds() / 60.0)
                    if temp <= delta:
                        htm_count += 1
                htm_id_list_no.append(htm_count)  

        htm_complete_yes.append(np.array(htm_id_list_yes, dtype = 'f'))
        htm_complete_no.append(np.array(htm_id_list_no, dtype = 'f'))

    return htm_complete_yes, htm_complete_no

In [67]:
def summary_output(delta):
    output_delta_yes, output_delta_no = random_puff(delta)
    agg_sum_delta_no = np.asarray([np.sum(arr) for arr in output_delta_no])
    agg_mean_delta_no  = np.asarray([np.mean(arr) for arr in output_delta_no])
    agg_mean_delta_no  = agg_mean_delta_no[~np.isnan(agg_mean_delta_no)]
    agg_count_delta_no  = np.asarray([len(arr) for arr in output_delta_no])
    agg_zero_delta_no  = np.asarray([np.count_nonzero(arr==0) for arr in output_delta_no], dtype='f')
    ind_zero_delta_no = np.divide(agg_zero_delta_no, agg_count_delta_no)
    ind_zero_delta_no = ind_zero_delta_no[~np.isnan(ind_zero_delta_no)]

    aggregate_frac_delta_no = np.divide(np.sum(agg_sum_delta_no),np.sum(agg_count_delta_no))
    aggregate_fraczero_delta_no = np.divide(np.sum(agg_zero_delta_no),np.sum(agg_count_delta_no))

    agg_sum_delta_yes = np.asarray([np.sum(arr) for arr in output_delta_yes])
    agg_mean_delta_yes  = np.asarray([np.mean(arr) for arr in output_delta_yes])
    agg_mean_delta_yes = agg_mean_delta_yes[~np.isnan(agg_mean_delta_yes)]
    agg_count_delta_yes  = np.asarray([len(arr) for arr in output_delta_yes])
    agg_zero_delta_yes  = np.asarray([np.count_nonzero(arr==0) for arr in output_delta_yes], dtype='f')
    ind_zero_delta_yes = np.divide(agg_zero_delta_yes, agg_count_delta_yes)
    ind_zero_delta_yes = ind_zero_delta_yes[~np.isnan(ind_zero_delta_yes)]

    aggregate_frac_delta_yes = np.divide(np.sum(agg_sum_delta_yes),np.sum(agg_count_delta_yes))
    aggregate_fraczero_delta_yes = np.divide(np.sum(agg_zero_delta_yes),np.sum(agg_count_delta_yes))

    print '% s minute window:' % (delta)
    print 'Prior to EMA Response: No'
    print 'Mean number of HTMGs prior (aggregated data): %s' % (np.round(aggregate_frac_delta_no,3))
    print 'Avg number of HTMGs (avg of means across indidivuals): %s' % (np.round(np.mean(agg_mean_delta_no),3))
    print 'Std dev of number of HTMGs (of means across indidivuals): %s' %  (np.round(np.std(agg_mean_delta_no),3))
    print 'Fraction of time no HTMGs in window (aggregated data): %s' %  (np.round(np.sum(aggregate_fraczero_delta_no),3))
    print 'Avg fraction of time no HTMGs in window (of fractions across individuals): %s' %  (np.round(np.mean(ind_zero_delta_no),3))
    print 'Std dev of fraction of time no HTMGs in window (of fractions across individuals): %s' %  (np.round(np.std(ind_zero_delta_no),3))
    print
    '''
    print '% s minute window:' % (delta)
    print 'Prior to EMA Response: Yes'
    print 'Mean number of HTMGs prior (aggregated data): %s' % (np.round(aggregate_frac_delta_yes,3))
    print 'Avg number of HTMGs (avg of means across indidivuals): %s' % (np.round(np.mean(agg_mean_delta_yes),3))
    print 'Std dev of number of HTMGs (of means across indidivuals): %s' %  (np.round(np.std(agg_mean_delta_yes),3))
    print 'Fraction of time no HTMGs in window (aggregated data): %s' %  (np.round(np.sum(aggregate_fraczero_delta_yes),3))
    print 'Avg fraction of time no HTMGs in window (of fractions across individuals): %s' %  (np.round(np.mean(ind_zero_delta_yes),3))
    print 'Std dev of fraction of time no HTMGs in window (of fractions across individuals): %s' %  (np.round(np.std(ind_zero_delta_yes),3))
    print
    '''

    return None


In [69]:
'''
Calculate the HTMG coverage for 5, 15, 30, and 60 minutes
around the event time.
'''

summary_output(5)

summary_output(15)

summary_output(30)

summary_output(60)

summary_output(90)

summary_output(120)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in divide
  
/Library/Python/2.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in divide


5 minute window:
Prior to EMA Response: No
Mean number of HTMGs prior (aggregated data): 0.688
Avg number of HTMGs (avg of means across indidivuals): 0.665
Std dev of number of HTMGs (of means across indidivuals): 0.578
Fraction of time no HTMGs in window (aggregated data): 0.626
Avg fraction of time no HTMGs in window (of fractions across individuals): 0.656
Std dev of fraction of time no HTMGs in window (of fractions across individuals): 0.204

15 minute window:
Prior to EMA Response: No
Mean number of HTMGs prior (aggregated data): 2.124
Avg number of HTMGs (avg of means across indidivuals): 1.946
Std dev of number of HTMGs (of means across indidivuals): 1.273
Fraction of time no HTMGs in window (aggregated data): 0.343
Avg fraction of time no HTMGs in window (of fractions across individuals): 0.369
Std dev of fraction of time no HTMGs in window (of fractions across individuals): 0.195

30 minute window:
Prior to EMA Response: No
Mean number of HTMGs prior (aggregated data): 4.188
A

In [213]:
''' 
Compute an anova decomposition using the poisson likelihood
This will test if there are significant differences across
individuals.
'''

llik_delta = 0; agg_llik_delta = 0
output_delta = contingent_puff(delta)
output_delta = np.asarray(output_delta)
agg_sum_delta = np.asarray([np.sum(arr) for arr in output_delta])
agg_mean_delta  = np.asarray([np.mean(arr) for arr in output_delta])
agg_count_delta  = np.asarray([len(arr) for arr in output_delta])
aggregate_frac_delta  = np.divide(np.sum(agg_sum_delta),np.sum(agg_count_delta))

for i in range(0, agg_mean_delta.size):
    user_mean = agg_mean_delta[i]
    row = output_delta[i]
    if user_mean > 0.0:
        llik_delta += np.sum(np.subtract(np.multiply(row, np.log(user_mean)),user_mean))
        agg_llik_delta += np.sum(np.subtract(np.multiply(row, np.log(aggregate_frac_delta)),aggregate_frac_delta))

D_delta = -2*agg_llik_delta + 2*llik_delta
print D_delta


from scipy.stats import chi2
n = np.sum(agg_count_delta)
k = output_delta.shape[0]
df = k-1
print k

print 'ANOVA p-value for current hour: %s' % (1-chi2.cdf(D_delta, df))

483.30132246017456
31
ANOVA p-value for current hour: 0.0
